## In this assignment, we will explore, segment, and cluster the neighborhoods in the city of Toronto. 

### For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset. 
### Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto

### Import necessary libraries

In [26]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import json
from pandas.io.json import json_normalize
import numpy as np

### Setup BeautifulSoup to scrape data from Wiki for Toronto data

In [27]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

resp = requests.get(url)
soup = BeautifulSoup(resp.text,"html.parser")

### Data preprocessing

In [28]:
unfilter_table = soup.find_all("table",{"class":"wikitable sortable"})

In [29]:
unfilter_table = str(unfilter_table).split("\n")

In [30]:
unfilter_header = soup.find_all("th")

In [31]:
header = ["","",""]
for counter, i in enumerate(unfilter_header):
    temp=str(i)
    header[counter] = (temp.split("/")[0].split(">")[1].split("<")[0].split("\n")[0])
    header[counter].replace("\n","")
    if counter == 2:
        break

In [32]:
content = []
for temp2 in unfilter_table:
    temp3 = str(temp2)
    if "<td>" in temp3:
        if "href" in temp3:
            content.append(temp3.split(">")[2].split("<")[0])
        else:
            content.append(temp3.split(">")[1].split("<")[0])

In [33]:
list1 = []
list2 = []
list3 = []

for count in range(0,int(len(content)),3):
    
    list1.append(content[count])
    list2.append(content[count+1])
    list3.append(content[count+2])

temp_dict = {
        header[0]:list1,
        header[1]:list2,
        header[2]:list3,
    }


### Move the data to Dataframe

In [34]:
df=pd.DataFrame(temp_dict)

In [35]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Drop the Not assigned in Borough column

In [36]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Combine the Neighbourhood with the same Postcode

In [37]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [38]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [39]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

### Set Neighbourhood = Borough if Not assigned

In [40]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [41]:
df.shape

(103, 3)

### Create new columns for Latitude and Longitude

In [42]:
df['Latitude'] = ""
df['Longitude'] = ""

### Read geo coordinates from csv

In [43]:
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [44]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Move the coordinates to Dataframe

In [45]:
for index, postal in enumerate(df['Postcode']):
    df.iloc[index,3] = float(geo.loc[geo['Postal Code'] == postal].Latitude)
    df.iloc[index,4] = float(geo.loc[geo['Postal Code'] == postal].Longitude)

df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [46]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### Visualize the neighborhoods 

In [47]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [48]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### The following steps follow the same analysis as the New York Neighborhood Assignment 

In [49]:
CLIENT_ID = '1H41HY33PXOLBSF3RCY1KNBAJ4VEQCBFCWS5YTG3A5SD32WW' # your Foursquare ID
CLIENT_SECRET = 'REJRXYTHCAKF1QUTJMGZZGWW25FZKRTBI5V5DD3ITQXKMVVV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
CLIENT_SECRET:Rxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [50]:
df.loc[0, 'Neighbourhood']

'Rouge,Malvern'

In [51]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [77]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 400 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1H41HY33PXOLBSF3RCY1KNBAJ4VEQCBFCWS5YTG3A5SD32WW&client_secret=REJRXYTHCAKF1QUTJMGZZGWW25FZKRTBI5V5DD3ITQXKMVVV&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=400&limit=100'

In [78]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7ba034db1d81002c206c72'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8102863036, 'lng': -79.18937434458434},
   'sw': {'lat': 43.80308629639999, 'lng': -79.19933245541569}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display

In [79]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [80]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [81]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [83]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [84]:
print(toronto_venues.shape)
toronto_venues.head()

(2260, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [85]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56


In [86]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [87]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
toronto_onehot.shape

(2260, 275)

In [89]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,...,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.043478,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [90]:
toronto_grouped.shape

(100, 275)

In [91]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2   Steakhouse  0.04
3          Bar  0.04
4        Hotel  0.03


----Agincourt----
                 venue  freq
0       Breakfast Spot  0.25
1               Lounge  0.25
2       Sandwich Place  0.25
3         Skating Rink  0.25
4  Monument / Landmark  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2  Molecular Gastronomy Restaurant   0.0
3                      Music Venue   0.0
4                           Museum   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1            Beer Store  0.11
2  Fast Food Restaurant  0.11
3   Fried Chicken Joint  0.11
4           Coffee Shop  0.11


----Alderwood,Long Branch----
            ve

                  venue  freq
0        Clothing Store  0.16
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.08
3            Food Court  0.03
4              Tea Room  0.03


----First Canadian Place,Underground city----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.08
2        Hotel  0.05
3   Steakhouse  0.04
4   Restaurant  0.04


----Flemingdon Park,Don Mills South----
                 venue  freq
0     Asian Restaurant  0.08
1           Beer Store  0.08
2          Coffee Shop  0.08
3                  Gym  0.08
4  Japanese Restaurant  0.04


----Forest Hill North,Forest Hill West----
              venue  freq
0     Jewelry Store  0.25
1             Trail  0.25
2          Bus Line  0.25
3  Sushi Restaurant  0.25
4             Motel  0.00


----Glencairn----
                 venue  freq
0  Japanese Restaurant  0.25
1                  Pub  0.25
2     Asian Restaurant  0.25
3                 Park  0.25
4                Motel  0.00


----Guildwood,Morningside,West Hi

                venue  freq
0         Coffee Shop  0.07
1                Café  0.06
2               Hotel  0.05
3          Restaurant  0.05
4  Italian Restaurant  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.04
2                Café  0.04
3  Seafood Restaurant  0.03
4        Cocktail Bar  0.03


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3  American Restaurant  0.05
4               Bakery  0.05


----The Annex,North Midtown,Yorkville----
                           venue  freq
0                    Coffee Shop  0.13
1                           Café  0.13
2                 Sandwich Place  0.13
3                    Pizza Place  0.09
4  Vegetarian / Vegan Restaurant  0.04


----The Beaches----
               venue  freq
0  Health Food Store  0.25
1                Pub  0.25
2       Neighborhood  0.25
3             

In [92]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [93]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Cosmetics Shop,Hotel,American Restaurant,Burger Joint,Restaurant
1,Agincourt,Breakfast Spot,Sandwich Place,Lounge,Skating Rink,Yoga Studio,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Beer Store,Coffee Shop,Pharmacy,Empanada Restaurant,Ethiopian Restaurant
4,"Alderwood,Long Branch",Pizza Place,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pool,Gym,Pharmacy,Empanada Restaurant,Ethiopian Restaurant


In [110]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [112]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.dropna(inplace=True)
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944,3.0,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605,4.0,Bar,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Doner Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887,0.0,Pizza Place,Rental Car Location,Breakfast Spot,Medical Center,Mexican Restaurant,Electronics Store,Intersection,Creperie,Donut Shop,Concert Hall
3,M1G,Scarborough,Woburn,43.771,-79.2169,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,0.0,Caribbean Restaurant,Bakery,Bank,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Eastern European Restaurant,Donut Shop,Drugstore


In [113]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster_new = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster_new-1],
        fill=True,
        fill_color=rainbow[cluster_new-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [114]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,Pizza Place,Rental Car Location,Breakfast Spot,Medical Center,Mexican Restaurant,Electronics Store,Intersection,Creperie,Donut Shop,Concert Hall
3,Scarborough,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
4,Scarborough,0.0,Caribbean Restaurant,Bakery,Bank,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Eastern European Restaurant,Donut Shop,Drugstore
6,Scarborough,0.0,Discount Store,Department Store,Bus Station,Coffee Shop,Convenience Store,Hobby Shop,Field,Festival,Fast Food Restaurant,Farmers Market
7,Scarborough,0.0,Bakery,Bus Line,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Soccer Field,Park,Cupcake Shop,Dumpling Restaurant
8,Scarborough,0.0,Motel,Movie Theater,American Restaurant,Yoga Studio,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
9,Scarborough,0.0,Café,College Stadium,General Entertainment,Skating Rink,Yoga Studio,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
10,Scarborough,0.0,Indian Restaurant,Pet Store,Chinese Restaurant,Vietnamese Restaurant,Latin American Restaurant,Thrift / Vintage Store,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore
11,Scarborough,0.0,Middle Eastern Restaurant,Breakfast Spot,Auto Garage,Smoke Shop,Sandwich Place,Shopping Mall,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
12,Scarborough,0.0,Breakfast Spot,Sandwich Place,Lounge,Skating Rink,Yoga Studio,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


In [115]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,1.0,Playground,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Comfort Food Restaurant
48,Central Toronto,1.0,Playground,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Comfort Food Restaurant


In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,2.0,Playground,Park,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
23,North York,2.0,Park,Convenience Store,Bank,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
25,North York,2.0,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
30,North York,2.0,Playground,Airport,Park,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
40,East York,2.0,Park,Convenience Store,Rental Car Location,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
44,Central Toronto,2.0,Park,Swim School,Bus Line,Yoga Studio,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
50,Downtown Toronto,2.0,Park,Building,Trail,Playground,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
74,York,2.0,Park,Women's Store,Fast Food Restaurant,Market,Hakka Restaurant,Field,Harbor / Marina,Farmers Market,Falafel Restaurant,Event Space
98,York,2.0,Park,Convenience Store,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
100,Etobicoke,2.0,Pizza Place,Park,Bus Line,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


In [117]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space


In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4.0,Bar,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Doner Restaurant
80,York,4.0,Discount Store,Sandwich Place,Restaurant,Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
